In [1]:
import pickle
import pandas as pd
import geopandas as gpd

In [2]:
with open('colonies_bbox_psi_popdensity.pkl', 'rb') as f:
    colonies = pickle.load(f)

In [3]:
colonies.columns

Index(['AREA', 'USO_AREA_U', 'HOUSETAX_C', 'USO_FINAL', 'geometry', 'area_km2',
       'canal', 'railway', 'drain', 'barrier', 'centroid', 'ndmc_dist_km',
       'population', 'exclude_from_psi', 'nbrs_bbox', 'nbrs_dist_bbox',
       'bank_count', 'bank_pcen', 'bank_idx', 'health_count', 'health_pcen',
       'health_idx', 'police_count', 'police_pcen', 'police_idx',
       'ration_count', 'ration_pcen', 'ration_idx', 'school_count',
       'school_pcen', 'school_idx', 'transport_count', 'transport_pcen',
       'transport_idx', 'road_length', 'road_pcen', 'road_idx', 'unnorm_psi',
       'norm_psi'],
      dtype='object')

In [41]:
settlement_types = colonies['USO_FINAL'].unique()

settlement_nbrs = dict()

for settlement_type1 in settlement_types:
    settlement_nbrs[settlement_type1] = dict()
    
    for settlement_type2 in settlement_types:
        settlement_nbrs[settlement_type1][settlement_type2] = 0

In [43]:
#settlement_nbrs

In [44]:
#ten_colonies = colonies[:10]
#ten_colonies.iloc[0, :]

In [45]:
for idx, row in colonies.iterrows():
    polygon_settlement_type = row['USO_FINAL']
    
    #print(f'{polygon_settlement_type} is the settlement type for row {idx}')
    
    for uso_id in row['nbrs_bbox']:
        neighbor_row = colonies[colonies['USO_AREA_U'] == uso_id]
        
        neighbor_settlement_type = neighbor_row['USO_FINAL'].values[0]
        
        #print(f'{uso_id} has type {neighbor_settlement_type}')
        
        settlement_nbrs[polygon_settlement_type][neighbor_settlement_type] += 1
        
    #print("-------------------------------------")

In [107]:
neighbors_df = pd.DataFrame(columns=settlement_types)

In [108]:
for settlement_type in settlement_types:
    row_to_concat = pd.DataFrame(settlement_nbrs[settlement_type], index=[settlement_type])
    neighbors_df = pd.concat([neighbors_df, row_to_concat])

In [109]:
neighbors_df

,Planned,RV,RUAC,Industrial,UV,UAC,Other,SDA,JJR,JJC
Planned,3410,86,338,43,347,1007,78,74,82,545
RV,91,268,140,0,12,545,6,0,6,35
RUAC,330,127,637,11,63,1268,8,15,21,89
Industrial,91,2,9,12,8,32,1,6,3,170
UV,346,17,66,3,58,327,19,0,12,50
UAC,916,390,985,3,339,5089,27,3,44,127
Other,89,5,7,0,15,25,12,4,2,32
SDA,61,0,17,7,0,6,4,374,0,11
JJR,91,8,40,0,15,65,2,0,25,153
JJC,568,39,70,9,69,153,30,16,75,258


In [137]:
neighbors_df['Total'] = 0

In [138]:
neighbors_df

,Planned,RV,RUAC,Industrial,UV,UAC,Other,SDA,JJR,JJC,Total
Planned,3410,86,338,43,347,1007,78,74,82,545,0
RV,91,268,140,0,12,545,6,0,6,35,0
RUAC,330,127,637,11,63,1268,8,15,21,89,0
Industrial,91,2,9,12,8,32,1,6,3,170,0
UV,346,17,66,3,58,327,19,0,12,50,0
UAC,916,390,985,3,339,5089,27,3,44,127,0
Other,89,5,7,0,15,25,12,4,2,32,0
SDA,61,0,17,7,0,6,4,374,0,11,0
JJR,91,8,40,0,15,65,2,0,25,153,0
JJC,568,39,70,9,69,153,30,16,75,258,0


In [139]:
for settlement_type in neighbors_df.index:
    sum_of_neighbors = neighbors_df.loc[settlement_type].sum(0)
    neighbors_df.loc[settlement_type, 'Total'] = sum_of_neighbors

In [140]:
neighbors_df

,Planned,RV,RUAC,Industrial,UV,UAC,Other,SDA,JJR,JJC,Total
Planned,3410,86,338,43,347,1007,78,74,82,545,6010
RV,91,268,140,0,12,545,6,0,6,35,1103
RUAC,330,127,637,11,63,1268,8,15,21,89,2569
Industrial,91,2,9,12,8,32,1,6,3,170,334
UV,346,17,66,3,58,327,19,0,12,50,898
UAC,916,390,985,3,339,5089,27,3,44,127,7923
Other,89,5,7,0,15,25,12,4,2,32,191
SDA,61,0,17,7,0,6,4,374,0,11,480
JJR,91,8,40,0,15,65,2,0,25,153,399
JJC,568,39,70,9,69,153,30,16,75,258,1287


In [157]:
neighbors_df_percent = neighbors_df.copy()

In [158]:
for idx, row in neighbors_df_percent.iterrows():
    for settlement_type in settlement_types:
        pct = row[settlement_type]/row['Total']
        neighbors_df_percent.loc[idx, settlement_type] = pct

In [159]:
neighbors_df_percent

,Planned,RV,RUAC,Industrial,UV,UAC,Other,SDA,JJR,JJC,Total
Planned,0.567388,0.0143095,0.0562396,0.00715474,0.0577371,0.167554,0.0129784,0.0123128,0.0136439,0.0906822,6010
RV,0.0825023,0.242974,0.126927,0,0.0108794,0.494107,0.00543971,0,0.00543971,0.0317316,1103
RUAC,0.128455,0.0494356,0.247956,0.00428182,0.0245232,0.493577,0.00311405,0.00583885,0.00817439,0.0346438,2569
Industrial,0.272455,0.00598802,0.0269461,0.0359281,0.0239521,0.0958084,0.00299401,0.0179641,0.00898204,0.508982,334
UV,0.385301,0.018931,0.0734967,0.00334076,0.064588,0.364143,0.0211581,0,0.013363,0.0556793,898
UAC,0.115613,0.0492238,0.124322,0.000378644,0.0427868,0.642307,0.0034078,0.000378644,0.00555345,0.0160293,7923
Other,0.465969,0.026178,0.0366492,0,0.078534,0.13089,0.0628272,0.0209424,0.0104712,0.167539,191
SDA,0.127083,0,0.0354167,0.0145833,0,0.0125,0.00833333,0.779167,0,0.0229167,480
JJR,0.22807,0.0200501,0.100251,0,0.037594,0.162907,0.00501253,0,0.0626566,0.383459,399
JJC,0.441336,0.030303,0.0543901,0.00699301,0.0536131,0.118881,0.02331,0.012432,0.0582751,0.200466,1287


In [161]:
neighbors_df_percent = neighbors_df_percent.drop(columns=['Total'])

In [162]:
neighbors_df_percent

,Planned,RV,RUAC,Industrial,UV,UAC,Other,SDA,JJR,JJC
Planned,0.567388,0.0143095,0.0562396,0.00715474,0.0577371,0.167554,0.0129784,0.0123128,0.0136439,0.0906822
RV,0.0825023,0.242974,0.126927,0,0.0108794,0.494107,0.00543971,0,0.00543971,0.0317316
RUAC,0.128455,0.0494356,0.247956,0.00428182,0.0245232,0.493577,0.00311405,0.00583885,0.00817439,0.0346438
Industrial,0.272455,0.00598802,0.0269461,0.0359281,0.0239521,0.0958084,0.00299401,0.0179641,0.00898204,0.508982
UV,0.385301,0.018931,0.0734967,0.00334076,0.064588,0.364143,0.0211581,0,0.013363,0.0556793
UAC,0.115613,0.0492238,0.124322,0.000378644,0.0427868,0.642307,0.0034078,0.000378644,0.00555345,0.0160293
Other,0.465969,0.026178,0.0366492,0,0.078534,0.13089,0.0628272,0.0209424,0.0104712,0.167539
SDA,0.127083,0,0.0354167,0.0145833,0,0.0125,0.00833333,0.779167,0,0.0229167
JJR,0.22807,0.0200501,0.100251,0,0.037594,0.162907,0.00501253,0,0.0626566,0.383459
JJC,0.441336,0.030303,0.0543901,0.00699301,0.0536131,0.118881,0.02331,0.012432,0.0582751,0.200466


In [163]:
neighbors_df.to_csv('neighbors_counts.csv')

In [164]:
neighbors_df_percent.to_csv('neighbors_pct.csv')